In [1]:
"""Print all objects and runs in a copick project."""

import copick

# Initialize the root object from a configuration file
root = copick.from_file("copick_config.json")

# List all available objects
obj_info = [(o.name, o.label) for o in root.pickable_objects]

print("Pickable objects in this project:")
for name, label in obj_info:
    print(f"  {name}: {label}")

# Execute a function on each run in the project
runs = root.runs

print("Runs in this project:")
for run in runs:
    print(f"Run: {run.name}")
    # Do something with the run

Pickable objects in this project:
  apo-ferritin: 1
  beta-amylase: 2
  beta-galactosidase: 3
  ribosome: 4
  thyroglobulin: 5
  virus-like-particle: 6
  membrane: 8
  background: 9
Runs in this project:
Run: TS_5_4
Run: TS_69_2
Run: TS_6_4
Run: TS_6_6
Run: TS_73_6
Run: TS_86_3
Run: TS_99_9


In [2]:
import os
import json
import zarr
import copick

# Paths
config_path = "copick_config.json"
data_root = "../data/train/static/ExperimentRuns"

# Load the configuration
with open(config_path, 'r') as f:
    config = json.load(f)

static_root = config["static_root"]
overlay_root = config["overlay_root"]

# Function to load CryoET OME-Zarr data at all resolutions
def load_cryoet_omezarr_data_all_resolutions(experiment, particle_types):
    zarr_path = os.path.join(static_root, f"ExperimentRuns/{experiment}/VoxelSpacing10.000/denoised.zarr")
    picks_path = os.path.join(overlay_root, f"ExperimentRuns/{experiment}/Picks")
    
    # Open the OME-Zarr dataset
    store = zarr.DirectoryStore(zarr_path)
    root = zarr.open(store, mode='r')

    # Check for multiscales metadata
    if "multiscales" not in root.attrs:
        raise ValueError("This Zarr file does not contain OME-Zarr multiscales metadata.")
    
    # Retrieve metadata for multiscale levels
    multiscales = root.attrs["multiscales"]
    datasets = multiscales[0]["datasets"]  # Assuming one multiscale entry

    # Load data at each resolution level
    resolution_data = {}
    for idx, dataset in enumerate(datasets):
        data_path = dataset["path"]
        print(f"Loading resolution level {idx} from path '{data_path}'...")
        resolution_data[f"level_{idx}"] = root[data_path][:]
    
    # Load ground truth JSONs (particle coordinates)
    particle_coords = {}
    for particle in particle_types:
        json_path = os.path.join(picks_path, f"{particle}.json")
        if os.path.exists(json_path):
            with open(json_path, 'r') as f:
                particle_coords[particle] = json.load(f)
        else:
            print(f"Ground truth file for {particle} not found.")
    
    return resolution_data, particle_coords

# Example usage
experiment_name = "TS_73_6"  # Replace with the actual experiment name
particle_types = [obj["name"] for obj in config["pickable_objects"] if obj["is_particle"]]

# Load dataset
tomographic_data_by_resolution, ground_truth = load_cryoet_omezarr_data_all_resolutions(experiment_name, particle_types)

# Display loaded data
for resolution, tomo_data in tomographic_data_by_resolution.items():
    print(f"Resolution {resolution}: data shape = {tomo_data.shape}")

print("Loaded particles and coordinates:")
for name, data in ground_truth.items():
    num_coords = len(data["points"])
    print(f"  {name}: {num_coords} coordinates loaded")

Loading resolution level 0 from path '0'...
Loading resolution level 1 from path '1'...
Loading resolution level 2 from path '2'...
Resolution level_0: data shape = (184, 630, 630)
Resolution level_1: data shape = (92, 315, 315)
Resolution level_2: data shape = (46, 158, 158)
Loaded particles and coordinates:
  apo-ferritin: 95 coordinates loaded
  beta-amylase: 12 coordinates loaded
  beta-galactosidase: 14 coordinates loaded
  ribosome: 46 coordinates loaded
  thyroglobulin: 28 coordinates loaded
  virus-like-particle: 22 coordinates loaded


In [3]:
import numpy as np
coordinates_array = np.array([[p['location']['x'], p['location']['y'], p['location']['z']] 
                              for v in ground_truth.values() for p in v['points']])

In [4]:
print(coordinates_array[:,0].max())
print(coordinates_array[:,1].max())
print(coordinates_array[:,2].max())

6183.204
6098.949
1335.552


In [5]:
import torchio as tio
import numpy as np

def preprocess_and_augment(volume, coordinates, target_resolution=(1.0, 1.0, 1.0)):
    """
    Preprocess and augment a 3D volume with a list of coordinates using TorchIO.
    
    Parameters:
        volume (np.ndarray): Input 3D volume.
        coordinates (np.ndarray): Array of 3D coordinates (N x 3).
        target_resolution (tuple): Target resolution for resampling (default is placeholder).

    Returns:
        augmented_volume (np.ndarray): Augmented volume.
        transformed_coordinates (np.ndarray): Transformed 3D coordinates.
    """
    # Ensure coordinates are a numpy array
    coordinates = np.asarray(coordinates)
    
    # Normalize the volume
    volume_tensor = tio.ScalarImage(tensor=volume[np.newaxis, ...])
    volume_normalized = tio.transforms.ZNormalization()(volume_tensor)
    
    # Resample to the target resolution
    resample_transform = tio.Resample(target_resolution)
    volume_resampled = resample_transform(volume_normalized)
    coordinates = resample_transform.apply_to_points(coordinates)
    
    # Calculate new dimensions for zero-padding
    original_shape = volume_resampled.shape[1:]  # Exclude the channel axis
    max_dim = int(np.ceil(np.sqrt(2) * max(original_shape)))
    padded_shape = (max_dim, max_dim, max_dim)
    
    # Center the volume and coordinates with zero-padding
    pad_transform = tio.CropOrPad(padded_shape)
    volume_padded = pad_transform(volume_resampled)
    coordinates = pad_transform.apply_to_points(coordinates)
    
    # Apply random flip
    flip_transform = tio.RandomFlip(axes=(0, 1, 2))
    volume_flipped = flip_transform(volume_padded)
    coordinates = flip_transform.apply_to_points(coordinates)
    
    # Apply random rotation
    rotation_transform = tio.RandomAffine(scales=1, degrees=(0, 45), isotropic=False)
    volume_rotated = rotation_transform(volume_flipped)
    coordinates = rotation_transform.apply_to_points(coordinates)
    
    # Extract the final augmented volume as a numpy array
    augmented_volume = volume_rotated.tensor.numpy()[0]  # Remove channel axis
    
    return augmented_volume, coordinates

In [7]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact

# Assuming `volume` is your 3D data array (replace with your actual data volume)
# coordinates_array is extracted from the earlier numpy code
level = 0

volume = tomographic_data_by_resolution[f'level_{level}']

scale = 10 * (2**level)

adjusted_coords = (np.array(coordinates_array)[:, [2, 1, 0]] ) / scale


def plot_cross_section(i):
    # Determine slicing planes (current x, y, z coordinates for index)
    
    coords = adjusted_coords
    x_coord = int(coords[i, 0])
    y_coord = int(coords[i, 1])
    z_coord = int(coords[i, 2])
    
    # Create 3D cross-section plots with only the current coordinate visualized
    plt.figure(figsize=(15, 5))

    # Slice at x-coordinate
    plt.subplot(131)
    plt.imshow(volume[x_coord, :, :], cmap="viridis")
    plt.scatter(z_coord, y_coord, color='red', s=2)  # Map y, z in this slice
    plt.title(f'Slice at x={x_coord}')

    # Slice at y-coordinate
    plt.subplot(132)
    plt.imshow(volume[:, y_coord, :], cmap="viridis")
    plt.scatter(z_coord, x_coord, color='red', s=2)  # Map x, z in this slice
    plt.title(f'Slice at y={y_coord}')

    # Slice at z-coordinate
    plt.subplot(133)
    plt.imshow(volume[:, :, z_coord], cmap="viridis")
    plt.scatter(y_coord, x_coord, color='red', s=2)  # Map x, y in this slice
    plt.title(f'Slice at z={z_coord}')

    plt.show()

# Interactive Slider for scrolling through slices
interact(plot_cross_section, i=(0, len(coordinates_array) - 1))

interactive(children=(IntSlider(value=108, description='i', max=216), Output()), _dom_classes=('widget-interac…

<function __main__.plot_cross_section(i)>